In [6]:
#importing the required libraries
import os
import numpy as np
import pandas as pd
import datetime
import time
import dateutil
from selenium import webdriver
from selenium.webdriver.chrome.options import Options 
from bs4 import BeautifulSoup as bs

In [11]:
#getting the required dates 
now = datetime.datetime.today()
from_date = now + dateutil.relativedelta.relativedelta(months = -2)
from_date = from_date.strftime('%d/%m/%Y')
to_date = now + dateutil.relativedelta.relativedelta(months = -1)
to_date = to_date.strftime('%d/%m/%Y')

## Crawler

In [3]:
options = Options()
options.add_argument('--start-maximized')
#options.add_argument('--headless')
driver = webdriver.Chrome(options = options)

#crowler
driver.get('http://rubberboard.org.in/public')
time.sleep(5)

#click on daily link
driver.find_element_by_xpath('//*[@id="tabinfo"]/div[3]/div[2]/div/div[5]/a[1]').click()
time.sleep(10)

# inserting from date 
driver.find_element_by_xpath('//*[@id="txtFromDate"]').send_keys(from_date)

time.sleep(5)

#inserting to date
todate = driver.find_element_by_xpath('//*[@id="txtToDate"]').send_keys(to_date)

time.sleep(5)

#inserting the grade
driver.find_element_by_xpath('//*[@id="grade"]/option[2]').click()

# clicking on go
driver.find_element_by_xpath('//*[@id="dayForm"]/div/div[4]/button').click()

data = driver.page_source

driver.quit()

## Scrapper

In [12]:
soup = bs(data,'html.parser')
div = soup.find('div',{'id': 'Kottayam'})
tr= div.findAll('tr')

In [13]:
tr

[<tr>
 <th rowspan="2" style="font-weight:normal;"><i style="color:brown">RSS4</i></th><th colspan="2" style="font-weight:normal;">In ₹ </th><th colspan="2" style="font-weight:normal;">In US$ </th>
 </tr>, <tr>
 <th style="font-weight:normal;">Price </th><th style="font-weight:normal;">Change </th><th style="font-weight:normal;">Price </th><th style="font-weight:normal;">Change </th>
 </tr>, <tr class="">
 <td style="text-align: left;"><i class="fa fa-leaf"></i> 16-07-2020</td><td style="text-align: right;"><i class="fa fa-cube" style="color:brown"></i> <i style="color:brown">12900.0</i></td><td style="text-align: right;"><i style="color:brown">0.00</i></td><td style="text-align: right;"><i class="fa fa-caret-up" style="color:green"></i> <i style="color:brown">171.30</i></td><td style="text-align: right;"><i style="color:brown">0.19</i></td>
 </tr>, <tr class="">
 <td style="text-align: left;"><i class="fa fa-leaf"></i> 17-07-2020</td><td style="text-align: right;"><i class="fa fa-cube

In [14]:
filter = []
for rows in tr :
    cols = rows.findAll('td')
    cols = [ele.text.strip() for ele in cols]
    filter.append(cols)

filter


[[],
 [],
 ['16-07-2020', '12900.0', '0.00', '171.30', '0.19'],
 ['17-07-2020', '12900.0', '0.00', '171.30', '0.00'],
 ['18-07-2020', '13000.0', '100.00', '172.80', '1.49'],
 ['20-07-2020', '13200.0', '200.00', '175.75', '2.98'],
 ['21-07-2020', '13200.0', '0.00', '176.10', '0.32'],
 ['22-07-2020', '13200.0', '0.00', '176.55', '0.47'],
 ['23-07-2020', '13200.0', '0.00', '176.75', '0.19'],
 ['24-07-2020', '13100.0', '-100.00', '175.25', '-1.50'],
 ['25-07-2020', '13050.0', '-50.00', '174.30', '-0.93'],
 ['27-07-2020', '13000.0', '-50.00', '173.65', '-0.67'],
 ['28-07-2020', '13000.0', '0.00', '173.90', '0.25'],
 ['29-07-2020', '13000.0', '0.00', '173.90', '0.03'],
 ['30-07-2020', '13100.0', '100.00', '175.20', '1.29'],
 ['01-08-2020', '13100.0', '0.00', '175.05', '-0.17'],
 ['03-08-2020', '13200.0', '100.00', '176.55', '1.50'],
 ['04-08-2020', '13200.0', '0.00', '176.05', '-0.49'],
 ['05-08-2020', '13250.0', '50.00', '176.45', '0.41'],
 ['06-08-2020', '13300.0', '50.00', '177.55', '1.09

In [15]:
filter= filter[2:] # removing the empty rows
col = ['Date','Price (RS)','Change(RS)','Price($)','Change($)'] #naming the columns
kottayam = pd.DataFrame(columns=col,data=filter)

In [16]:
kottayam

,Date,Price (RS),Change(RS),Price($),Change($)
0,16-07-2020,12900.0,0.00,171.30,0.19
1,17-07-2020,12900.0,0.00,171.30,0.00
2,18-07-2020,13000.0,100.00,172.80,1.49
3,20-07-2020,13200.0,200.00,175.75,2.98
4,21-07-2020,13200.0,0.00,176.10,0.32
5,22-07-2020,13200.0,0.00,176.55,0.47
6,23-07-2020,13200.0,0.00,176.75,0.19
7,24-07-2020,13100.0,-100.00,175.25,-1.50
8,25-07-2020,13050.0,-50.00,174.30,-0.93
9,27-07-2020,13000.0,-50.00,173.65,-0.67
